## Set up and load model

In [1]:
import numpy as np
import random
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(0)

In [2]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
MODEL_DIR = 'content/models'
DATA_DIR = 'content/data'

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

model_id = "google/gemma-2-2b"
load_tokenizer_only = False

cache_dir = MODEL_DIR
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

if not load_tokenizer_only:
  model = AutoModelForCausalLM.from_pretrained(
      model_id, low_cpu_mem_usage=True, device_map='auto', cache_dir=cache_dir,
      torch_dtype=torch.bfloat16)
  model = model.eval()
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

VOCAB = sorted(tokenizer.vocab, key=tokenizer.vocab.get)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Load and edit datasets

In [6]:
# Load the city entites from Huggingface

from datasets import load_dataset

ravel_city_entities = load_dataset("hij/ravel", "city_entity")
ravel_city_prompts = load_dataset("hij/ravel", "city_prompt")

In [7]:
# For MIB, use only the country, continent, and language

# Remove other attributes from ravel_city_entities
from datasets import DatasetDict
columns_to_remove = ["Latitude", "Longitude", "Timezone", "URL"]
ravel_city_entities = ravel_city_entities.map(lambda x: x, remove_columns=columns_to_remove)

# Remove other attributes from ravel_city_prompts
attributes_to_remove = {"Latitude", "Longitude", "Timezone"}
ravel_city_prompts = ravel_city_prompts.filter(lambda x: x["Attribute"] not in attributes_to_remove)

In [8]:
import collections

# Group prompts by attribute.
attribute_prompts = collections.defaultdict(list)
prompt_splits = {}
for split in ravel_city_prompts:
  for prompt in ravel_city_prompts[split]:
    attribute_prompts[prompt['Attribute']].append(prompt['Template'])
    prompt_splits[prompt['Template']] = split
attribute_prompts = dict(attribute_prompts)

# Build entity to attributes mapping.
attributes = ['Country', 'Continent', 'Language'] #, 'Latitude', 'Longitude', 'Timezone']
entity_splits = {}
entity_attributes = collections.defaultdict(list)
for split in ravel_city_entities:
  for entity in ravel_city_entities[split]:
    entity_attributes[entity['City']].append({k: v for k, v in entity.items() if k in attributes})
    entity_splits[entity['City']] = split
entity_attributes = dict(entity_attributes)

In [9]:
import json
import os

entity_type = 'city'

print(f'#entities={len(entity_attributes)}, #prompt_templates={sum(map(len, attribute_prompts.values()))}')

prompts_to_meta_data = {t % x: {'entity': x, 'attr': a, 'template': t}
               for x in entity_attributes
               for a, ts in attribute_prompts.items()
               for t in ts
               # An empty attribute means the prompt is from Wikipedia, which
               # does not query for a specific attribute.
               if a != ''}
print(len(prompts_to_meta_data))

In [52]:
# from utils.generation_utils import generate_batched

# prompt_max_length = 48

# prompt_to_output = generate_batched(
#     model,
#     tokenizer,
#     list(prompts_to_meta_data),
#     prompt_max_length+8,
#     prompt_max_length=prompt_max_length,
#     batch_size=64)
# prompt_to_output = {k: v[len(k):] for k, v in prompt_to_output}

# torch.save(prompt_to_output, os.path.join(DATA_DIR, 'ravel_gemma-2-2b_city_prompt_to_output.pt'))

In [10]:
# Directly load the pre-computed outputs.
prompt_to_output = torch.load(os.path.join(DATA_DIR, 'ravel_gemma-2-2b_city_prompt_to_output.pt'))
len(prompt_to_output)

/tmp/ipykernel_3615235/580774497.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prompt_to_output = torch.load(os.path.join(DATA_DIR, 'ravel_gemma-2-2b_city_prompt_to_ou

415226

In [ ]:
#@title Behavioral Test

# import collections
# import re
# import numpy as np


# from zoneinfo import ZoneInfo
# import datetime

# def timezone_name_to_utc_offset(name):
#   try:
#     offset =  ZoneInfo(name).utcoffset(datetime.datetime.now()).seconds
#   except:
#     return 'NOT_FOUND'
#   sign = '+'
#   if offset // 3600 >= 12:
#     offset = 24 * 3600 - offset
#     sign = '-'
#   fmt_offset = str(datetime.timedelta(seconds=offset)).rsplit(':', 1)[0]
#   if fmt_offset.startswith('0') and offset >= 1800:
#     fmt_offset = fmt_offset[1:]
#   return f'{sign}{fmt_offset}'


# sorted_entity = sorted(set([v['entity'] for v in prompts_to_meta_data.values()]))
# sorted_template = sorted(set([v['template'] for v in prompts_to_meta_data.values()]))
# stats = np.zeros([len(sorted_entity), len(sorted_template)])
# for p, out in prompt_to_output.items():
#   attr = prompts_to_meta_data[p]['attr']
#   entity = prompts_to_meta_data[p]['entity']
#   # Each entity might be mapped to multiple attribute values.
#   label = '|'.join(set([x[attr] for x in entity_attributes[entity] if x[attr]]))
#   if not label:
#     continue
#   norm_label = label.lower()
#   norm_out = out.split('"')[0].strip(' "').replace('\\/', '/').lower()
#   if len(norm_label) < len(norm_out):
#     correct = int(norm_out.startswith(norm_label))
#   else:
#     correct = int(norm_label.startswith(norm_out))

  # Exceptions
#   if re.search('coord|"lat"|"long"|latitude|coordinates|longitude', p):
#     try:
#       correct = int((float(norm_label.strip('-−')) - float(re.findall(r'\d+', norm_out)[0])) <= 2)
#     except:
#       correct = 0
  # if re.search('United States|United Kingdom', label):
  #   norm_label = label.strip().replace('the ', '')
  #   norm_out = out[len(p):].strip().replace('the ', '')
  #   correct = int(norm_out.startswith(norm_label) or norm_out.startswith('England'))
  # if re.search('South Korea', label):
  #   correct = int(norm_out.startswith('korea') or norm_out.startswith('south korea'))
  # if re.search('North America', label):
  #   correct = norm_label in norm_out or norm_out == 'na' or norm_out.startswith('america')
  # if re.search('Mandarin', label):
  #   correct = norm_out in norm_label or norm_out == 'chinese'
  # if re.search('language', p) and ',' in norm_label:
  #   correct = any(lang in norm_out for lang in norm_label.split(','))
#   if re.search('UTC', p) and '/' in norm_label:
#     norm_label = [timezone_name_to_utc_offset(l) for l in label.split('|')]
#     correct = any(norm_out.startswith(l.split(':')[0]) for l in label.split('|'))
#     if not correct and re.search(r'[+\-]0\d', norm_out):
#       correct = any(norm_out.replace('0', '', 1).startswith(l.split(':')[0]) for l in norm_label)
#     norm_label = '|'.join(norm_label)
#     # Summer daylight saving time
#     if not correct and (
#         re.search(r'\-[5-8]', norm_label) and label.startswith('America') or
#         re.search(r'\+[0-3]', norm_label) and label.startswith('Europe') or
#         re.search(r'\+[0-3]', norm_label) and label.startswith('Africa')):
#       #print('SUMMER TIME:', norm_label, norm_out)
#       out_offset_match = re.search(r'[+\-]?(\d\d?):\d+', norm_out)
#       label_offset_match = re.search(r'[+\-]?(\d\d?):\d+', norm_label)
#       if out_offset_match and label_offset_match:
#         norm_out_offset = int(out_offset_match.group(1))
#         norm_label_offset = int(label_offset_match.group(1))
#         correct = (norm_out_offset <= norm_label_offset + 1 and
#                    norm_out_offset >= norm_label_offset - 1)
#     if not correct and re.search(r'[+\-](\d+)', norm_out) and int(
#         re.search(r'[+\-](\d+)', norm_out).group(1)) > 11:
#       offset = 24 - int(re.search(r'[+\-](\d+)', norm_out).group(1))
#       correct = str(offset) in norm_label
#   stats[sorted_entity.index(prompts_to_meta_data[p]['entity']), sorted_template.index(prompts_to_meta_data[p]['template'])] += int(correct)

# print('-----------------------------------')
# for i in np.argsort(stats.sum(axis=0))[::-1]:
#   print(sorted_template[i], int(stats[:, i].sum()), len(stats[:, i]))
# for i in np.argsort(stats.sum(axis=-1))[::-1]:
#   print(sorted_entity[i], int(stats[i].sum()), len(stats[i]))

# # Keep the top K entities and templates.
# num_entity = 800
# num_template = 200
# kept_entity_index = np.argsort(stats.sum(axis=1))[-num_entity:]
# KEPT_ENTITY = [sorted_entity[i] for i in kept_entity_index]
# topk_template_index = set(np.argsort(stats.sum(axis=0))[-num_template:])
# kept_template_index = []
# # A dict of all kept attribute to prompts.
# KEPT_ATTR_TO_PROMPT_AND_SPLIT = {}
# for attr in attribute_prompts:
#   if not attr:
#     # Wikipedia prompts.
#     continue
#   # Kept the top 4 to 12 templates per attribute.
#   attr_indices = [sorted_template.index(t) for t in attribute_prompts[attr]]
#   per_attr_kept_template_index = sorted(attr_indices, key=lambda i: stats[:, i].sum())[-12:][::-1]
#   per_attr_kept_template_index = [x for i, x in enumerate(per_attr_kept_template_index)
#                                   if x in topk_template_index or i < 4]
#   kept_template_index.extend(per_attr_kept_template_index)
#   KEPT_ATTR_TO_PROMPT_AND_SPLIT[attr] = {sorted_template[i]: prompt_splits[sorted_template[i]]
#                                for i in per_attr_kept_template_index}
# print('Kept %d entity, %d prompt template' % (len(kept_entity_index), len(kept_template_index)))

# display('Average accuracy: %.2f%%' % (100 *  (stats[:, kept_template_index][kept_entity_index, :]).sum()/ (len(kept_entity_index) * len(kept_template_index))))

'Average accuracy: 99.10%'

In [11]:
# Simply checking model accuracy without filtering incorrect instances
# (For MIB, no filtering in the data generation stage)
import collections
import re
import numpy as np
from zoneinfo import ZoneInfo
import datetime

def timezone_name_to_utc_offset(name):
  try:
    offset =  ZoneInfo(name).utcoffset(datetime.datetime.now()).seconds
  except:
    return 'NOT_FOUND'
  sign = '+'
  if offset // 3600 >= 12:
    offset = 24 * 3600 - offset
    sign = '-'
  fmt_offset = str(datetime.timedelta(seconds=offset)).rsplit(':', 1)[0]
  if fmt_offset.startswith('0') and offset >= 1800:
    fmt_offset = fmt_offset[1:]
  return f'{sign}{fmt_offset}'

sorted_entity = sorted(set([v['entity'] for v in prompts_to_meta_data.values()]))
sorted_template = sorted(set([v['template'] for v in prompts_to_meta_data.values()]))
stats = np.zeros([len(sorted_entity), len(sorted_template)])

for p, out in prompt_to_output.items():
    attr = prompts_to_meta_data[p]['attr']
    entity = prompts_to_meta_data[p]['entity']
    label = '|'.join(set([x[attr] for x in entity_attributes[entity] if x[attr]]))
    if not label:
        continue
    norm_label = label.lower()
    norm_out = out.split('"')[0].strip(' "').replace('\\/', '/').lower()
    
    if len(norm_label) < len(norm_out):
        correct = int(norm_out.startswith(norm_label))
    else:
        correct = int(norm_label.startswith(norm_out))
    
    if re.search('United States|United Kingdom', label):
        norm_label = label.strip().replace('the ', '')
        norm_out = out[len(p):].strip().replace('the ', '')
        correct = int(norm_out.startswith(norm_label) or norm_out.startswith('England'))
    if re.search('South Korea', label):
        correct = int(norm_out.startswith('korea') or norm_out.startswith('south korea'))
    if re.search('North America', label):
        correct = norm_label in norm_out or norm_out == 'na' or norm_out.startswith('america')
    if re.search('Mandarin', label):
        correct = norm_out in norm_label or norm_out == 'chinese'
    if re.search('language', p) and ',' in norm_label:
        correct = any(lang in norm_out for lang in norm_label.split(','))
    stats[sorted_entity.index(prompts_to_meta_data[p]['entity']), sorted_template.index(prompts_to_meta_data[p]['template'])] += int(correct)

print('-----------------------------------')
for i in np.argsort(stats.sum(axis=0))[::-1]:
    print(sorted_template[i], int(stats[:, i].sum()), len(stats[:, i]))
for i in np.argsort(stats.sum(axis=-1))[::-1]:
    print(sorted_entity[i], int(stats[i].sum()), len(stats[i]))

# No filtering is applied. All entities and templates are retained.
kept_entity_index = list(range(len(sorted_entity)))  # Keeping all entities
KEPT_ENTITY = sorted_entity
kept_template_index = list(range(len(sorted_template)))  # Keeping all templates

KEPT_ATTR_TO_PROMPT_AND_SPLIT = {}
for attr in attribute_prompts:
    if not attr:
        continue
    attr_indices = [sorted_template.index(t) for t in attribute_prompts[attr]]
    KEPT_ATTR_TO_PROMPT_AND_SPLIT[attr] = {sorted_template[i]: prompt_splits[sorted_template[i]] for i in attr_indices}

print('Kept %d entity, %d prompt template' % (len(kept_entity_index), len(kept_template_index)))
print('Average accuracy: %.2f%%' % (100 * (stats[:, kept_template_index][kept_entity_index, :]).sum()/ (len(kept_entity_index) * len(kept_template_index))))


In [12]:
# Kept templates

attribute_prompts
for i in kept_template_index:
  print(f'{prompt_splits[sorted_template[i]]}\t{sorted_template[i]}\t{stats[:, i][kept_entity_index].mean():.2f}')

In [13]:
display(sum(map(len, KEPT_ATTR_TO_PROMPT_AND_SPLIT.values())))
for attr, prompt_to_split in KEPT_ATTR_TO_PROMPT_AND_SPLIT.items():
  display(attr, collections.Counter(prompt_to_split.values()))

133

'Country'

Counter({'train': 22, 'test': 13, 'val': 11})

'Continent'

Counter({'train': 22, 'test': 11, 'val': 10})

'Language'

Counter({'train': 20, 'val': 13, 'test': 11})

In [15]:
import json

ENTITY_TYPE = 'city'

# Filtered
KEPT_ENTITY_SPLITS = {e: entity_splits[e] for e in KEPT_ENTITY}
KEPT_PROMPT_SPLITS = {k: (a, v) for a, d in KEPT_ATTR_TO_PROMPT_AND_SPLIT.items() for k, v in d.items() if k.count('%') == 1}
# Wikipedia prompts
for prompt in attribute_prompts['']:
  KEPT_PROMPT_SPLITS[prompt] = ('Other', prompt_splits[prompt])
KEPT_ATTR_TO_PROMPT_AND_SPLIT = {k: {p: v for p, v in d.items() if p.count('%') == 1} for k, d in KEPT_ATTR_TO_PROMPT_AND_SPLIT.items()}
print(f'Total #entities={len(entity_attributes)} #attributes={len(KEPT_ATTR_TO_PROMPT_AND_SPLIT)} '
      f'#prompts={sum(map(len, attribute_prompts.values()))} #wiki_prompts={len(attribute_prompts[""])}')
print(f'Kept #entities={len(KEPT_ENTITY_SPLITS)} #prompts={len(KEPT_PROMPT_SPLITS)}')
for split in ('train', 'val', 'test'):
  print(split, f'Kept #entities={len([k for k, v in KEPT_ENTITY_SPLITS.items() if v == split])}',
               f'#prompts={len([k for k, v in KEPT_PROMPT_SPLITS.items() if v[1] == split])}')

In [16]:
from src.utils.generation_utils import generate_batched

wiki_prompts = [(t['Template'] % e)
                for s in ravel_city_prompts
                for t in ravel_city_prompts[s]
                for e in ([t['Entity']] if t['Entity']
                           else [a for a in KEPT_ENTITY_SPLITS if KEPT_ENTITY_SPLITS[a] == 'train' or s == 'train'])
                 ]
print(len(wiki_prompts))

# wiki_prompt_and_output = generate_batched(
#     model,
#     tokenizer,
#     wiki_prompts,
#     max_new_tokens=8,
#     batch_size=64)
# wiki_prompt_to_output = {k: v[len(k):] for k, v in wiki_prompt_and_output}

wiki_prompt_to_output = torch.load(os.path.join(DATA_DIR, 'ravel_wiki_city_prompt_to_output.pt'))

/tmp/ipykernel_3615235/2197762071.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wiki_prompt_to_output = torch.load(os.path.join(DATA_DIR, 'ravel_wiki_city_prompt_to_o

In [ ]:
# torch.save(wiki_prompt_to_output, os.path.join(DATA_DIR, 'ravel_wiki_city_prompt_to_output.pt'))

In [17]:
ALL_PROMPT_TO_OUTPUT = {**prompt_to_output, **wiki_prompt_to_output}

len(ALL_PROMPT_TO_OUTPUT)

472408

In [18]:
WIKI_PROMPT_SPLITS = {
    t['Template']: {'split': s, 'entity': t['Entity']}
    for s in ravel_city_prompts
    for t in ravel_city_prompts[s]}

In [19]:
import datasets
from datasets import Dataset
from src.utils.generate_ravel_instance import RAVELMetadata


def extract_label(text):
  """Extracts the first word or phrase from the text.

  The rules are hard-coded based on the model output values.
  You might want to update the rules when using different models or prompts.
  """
  tokens = re.split(r'(["]|[.,;]\s|\n| \(|\sand)', text + ' ')
  x = tokens[0]
  digit_match = re.search(r'\.\d\d', x)
  if digit_match:
      x = x[:digit_match.span(0)[1]]
  gender_match = re.match(r'\s?(his|her|himself|herself|she|he)[^\w]', x)
  if gender_match:
      x = x[:gender_match.span(1)[1]]
  if not x.strip():
      x = ' '.join(text.split(' ')[:2]).rstrip('.,"\n')
  assert x.strip()
  return x


def get_first_token(x):
  return re.split(r'[^\w\+\-]', x.strip(), re.UNICODE)[0]


def filter_inv_example(base_output, inv_output):
  different_outputs = (get_first_token(base_output) !=
                       get_first_token(inv_output))
  valid_outputs = (
      re.fullmatch(r'\s?[a-z0-9.:\-+]+', extract_label(base_output), re.IGNORECASE) and
      re.fullmatch(r'\s?[a-z0-9.:\-+]+', extract_label(inv_output), re.IGNORECASE))
  return len(inv_output) > 0 and valid_outputs and different_outputs


FEATURE_TYPES = datasets.Features({"input": datasets.Value("string"), "label": datasets.Value("string"),
                              "source_input": datasets.Value("string"), "source_label": datasets.Value("string"),
                              "inv_label": datasets.Value("string"),
                              'split': datasets.Value("string"), 'source_split': datasets.Value("string"),
                              'entity': datasets.Value("string"), 'source_entity': datasets.Value("string")})


ravel_metadata = RAVELMetadata(
    'gemma-2-2b',
    KEPT_ENTITY_SPLITS,
    KEPT_ATTR_TO_PROMPT_AND_SPLIT,
    KEPT_PROMPT_SPLITS,
    WIKI_PROMPT_SPLITS,
    ALL_PROMPT_TO_OUTPUT)

## Generate context test/val split

In [20]:
#@title Generate the Conetxt TEST/VAL Split

# Context Split: All entities are in TRAIN, but all prompts are in test/dev

import random

from src.utils.generate_ravel_instance import gen_context_test_split

TEST_TYPE = 'context'

# Take the first N examples only
first_n = 256

eval_split_to_raw_example = gen_context_test_split(
    ravel_metadata,
    extract_label_fn=extract_label,
    filter_example_fn=filter_inv_example,
    first_n=first_n)
eval_split_to_dataset = {
    split: Dataset.from_list(eval_split_to_raw_example[split][:first_n], features=FEATURE_TYPES)
    for split in eval_split_to_raw_example}

# Compute stats.
for split in eval_split_to_raw_example:
  print('\nSplit %s:\nTotal %d examples, kept first %d examples, %d unique input values,  %d unique entities, %d unique output values, %d unique output tokens' % (
      repr(split), len(eval_split_to_raw_example[split]), len(eval_split_to_dataset[split]),
      len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['input', 'source_input']])),
      len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['entity', 'source_entity']])),
      len(set([exp['inv_label'] for exp in eval_split_to_raw_example[split][:first_n]])),
      len(set([tokenizer(exp['inv_label']).input_ids[1] for exp in eval_split_to_raw_example[split][:first_n]]))))
  #for i, example in enumerate(eval_split_to_raw_example[split]):
  #  print(example)
  #  #print(tokenizer(example['input']).input_ids)
  #  break
  #for k in ('input', 'source_input'):
  #  input_ids = tokenizer(example[k])['input_ids']
  #  #print(k)
  #  #print(input_ids)
  #  print(list(zip([(32 - len(input_ids)) + i for i in range(len(input_ids))], tokenizer.batch_decode(input_ids))))
for split in ('test', 'val'):
  print(f'Split {split}: Total #subsplit={len([k for k in eval_split_to_raw_example if k.endswith(split)])} #Examples={sum(map(len, [v for k, v in eval_split_to_raw_example.items() if k.endswith(split)]))}')

In [22]:
# Merge subsplits
eval_split_to_raw_example_merged = collections.defaultdict(list)
for split in eval_split_to_raw_example:
  eval_split_to_raw_example_merged[re.sub(r'-causal|-output|-other', '', split)].extend(eval_split_to_raw_example[split])
eval_split_to_raw_example = dict(eval_split_to_raw_example_merged)

In [25]:
output_json_path = os.path.join(DATA_DIR, f'{ravel_metadata.instance}/{ravel_metadata.instance}_{ENTITY_TYPE}_{TEST_TYPE}_test.json')
print(output_json_path)
json.dump(eval_split_to_raw_example, open(output_json_path, 'w'), ensure_ascii=False)

## Generate entity test/val split

In [26]:
#@title Generate the Entity TEST/VAL Split

from src.utils.generate_ravel_instance import gen_entity_test_split

TEST_TYPE = 'entity'

# Take the first N examples only
first_n = 256

eval_split_to_raw_example = gen_entity_test_split(
    ravel_metadata,
    extract_label_fn=extract_label, filter_example_fn=filter_inv_example,
    first_n=first_n)

eval_split_to_dataset = {
    split: Dataset.from_list(eval_split_to_raw_example[split][:first_n], features=FEATURE_TYPES)
    for split in eval_split_to_raw_example}

# Stats
for split in eval_split_to_raw_example:
  print('Split %s: Total %d examples, kept first %d examples, %d unique input values,  %d unique entities, %d unique output values, %d unique output tokens' % (
      repr(split), len(eval_split_to_raw_example[split]), len(eval_split_to_dataset[split]),
      len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['input', 'source_input']])),
      len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['entity', 'source_entity']])),
      len(set([exp['inv_label'] for exp in eval_split_to_raw_example[split][:first_n]])),
      len(set([tokenizer(exp['inv_label']).input_ids[1] for exp in eval_split_to_raw_example[split][:first_n]]))))
  for i, example in enumerate(eval_split_to_raw_example[split]):
    print(example)
    #print(tokenizer(example['input']).input_ids)
    break
  for k in ('input', 'source_input'):
    input_ids = tokenizer(example[k])['input_ids']
    #print(k)
    #print(input_ids)
    print(list(zip([(32 - len(input_ids)) + i for i in range(len(input_ids))], tokenizer.batch_decode(input_ids))))
for split in ('test', 'val'):
  print(f'Split {split}: Total #subsplit={len([k for k in eval_split_to_raw_example if k.endswith(split)])} #Examples={sum(map(len, [v for k, v in eval_split_to_raw_example.items() if k.endswith(split)]))}')

In [27]:
# Merge subsplits
eval_split_to_raw_example_merged = collections.defaultdict(list)
for split in eval_split_to_raw_example:
  eval_split_to_raw_example_merged[re.sub(r'-causal|-output|-other', '', split)].extend(eval_split_to_raw_example[split])
eval_split_to_raw_example = dict(eval_split_to_raw_example_merged)

In [28]:
output_json_path = os.path.join(DATA_DIR, f'{ravel_metadata.instance}/{ravel_metadata.instance}_{ENTITY_TYPE}_{TEST_TYPE}_test.json')
print(output_json_path)
json.dump(eval_split_to_raw_example, open(output_json_path, 'w'), ensure_ascii=False)

## Generate train split

In [29]:
#@title Generate train split (for models that use counterfactuals)

import datasets
from datasets import Dataset

def gen_train_split(metadata, extract_label_fn, filter_example_fn, first_n=256):
  split_to_raw_example = {}
  # Group by attributes.
  target_split = 'train'
  for attr, prompt_to_split in metadata.attr_to_prompt.items():
      base_prompt_candiates = [p for p, s in prompt_to_split.items() if s == target_split]
      base_task_inputs = [
          ((prompt, entity), metadata.prompt_to_output[prompt % entity])
          for entity in metadata.get_entities(target_split)
          for prompt in random.sample(
              base_prompt_candiates, k=min(2, len(base_prompt_candiates)))]
      print(len(base_task_inputs))
      source_task_inputs = [
          ((source_prompt, entity), metadata.prompt_to_output[source_prompt % entity])
          for source_prompt, (source_attr, source_split) in KEPT_PROMPT_SPLITS.items()
          if source_split == target_split and source_attr != 'Other'
          for entity in metadata.sample_entities(target_split, k=1)
      ]
      wiki_source_task_inputs = [
          ((source_prompt, entity), metadata.prompt_to_output[source_prompt % entity])
          for source_prompt, split_and_arg in metadata.entity_prompt_to_split.items()
          if split_and_arg['split'] == target_split
          for entity in ([split_and_arg['entity']] if split_and_arg['entity']
                         else metadata.sample_entities(target_split, k=1))
      ]
      source_task_inputs = source_task_inputs + wiki_source_task_inputs
      if len(base_task_inputs) < 5 or len(source_task_inputs) < 5:
        continue
      print(attr, target_split, len(base_task_inputs), len(source_task_inputs), len(wiki_source_task_inputs))
      split_to_raw_example[f'{attr}-{target_split}'] = []
      for (p, a), v in base_task_inputs:
        source_input_candiates = [x for x in source_task_inputs if
                                  x[0][1] in ravel_metadata.entity_to_split and
                                  filter_example_fn(v, metadata.prompt_to_output[p % x[0][1]])]
        #print(len(source_input_candiates), v)
        split_to_raw_example[f'{attr}-{target_split}'].extend([{
          'input': p % a, 'label': extract_label_fn(v),
          'source_input': s_p % s_a, 'source_label': extract_label_fn(source_v),
          'inv_label': extract_label_fn(metadata.prompt_to_output[p % s_a]),
          'split': p, 'source_split': s_p,
          'entity': a, 'source_entity': s_a}
        for (s_p, s_a), source_v in random.sample(source_input_candiates, k=min(len(source_input_candiates), round(first_n / len(base_task_inputs))))
        if filter_example_fn(v, metadata.prompt_to_output[p % s_a]) and re.search('\w+', source_v)
      ])
  split_to_raw_example = {k: v for k, v in split_to_raw_example.items() if len(v) > 0}
  return split_to_raw_example


# Take the first N examples only
first_n = 40960

split_to_raw_example = gen_train_split(
    ravel_metadata,
    extract_label_fn=extract_label,
    filter_example_fn=filter_inv_example,
    first_n=first_n)

# Stats
for split in split_to_raw_example:
  print('Split %s: Total %d examples, kept first %d examples, %d unique input values,  %d unique entities, %d unique output values, %d unique output tokens' % (
      repr(split), len(split_to_raw_example[split]), len(split_to_raw_example[split]),
      len(set([exp[x] for exp in split_to_raw_example[split][:first_n] for x in ['input', 'source_input']])),
      len(set([exp[x] for exp in split_to_raw_example[split][:first_n] for x in ['entity', 'source_entity']])),
      len(set([exp['inv_label'] for exp in split_to_raw_example[split][:first_n]])),
      len(set([tokenizer(exp['inv_label']).input_ids[1] for exp in split_to_raw_example[split][:first_n]]))))
  for i, example in enumerate(split_to_raw_example[split]):
    print(example)
    break
for split in ('train',):
  print(f'Split {split}: Total #subsplit={len([k for k in split_to_raw_example if k.endswith(split)])} #Examples={sum(map(len, [v for k, v in split_to_raw_example.items() if k.endswith(split)]))}')

In [30]:
json_path = os.path.join(DATA_DIR, f'{ravel_metadata.instance}/{ravel_metadata.instance}_{ENTITY_TYPE}_train.json')
print(json_path)
json.dump(split_to_raw_example, open(json_path, 'w'), ensure_ascii=False)

## Preprocess labels

In [31]:
#@title Postprocess labels

import json
import re


entity_type = 'city'
instance =  'gemma-2-2b'


json_path = os.path.join(DATA_DIR, f'{instance}/{instance}_{entity_type}_context_test.json')
split_to_raw_example = json.load(open(json_path, 'r'))
print(len(split_to_raw_example))

all_labels = set()
for split in split_to_raw_example:
  for i in range(len(split_to_raw_example[split])):
    # if split.split('-')[0] in ['Latitude', 'Longitude'] or  split.split('-')[0] in attribute_prompts['Latitude'] or split.split('-')[0] in attribute_to_prompts['Longitude']:
    #   # Keep only the integer part.
    #   split_to_raw_example[split][i]['inv_label'] = split_to_raw_example[split][i]['inv_label'].replace('°', '.').split('.')[0]
    #   split_to_raw_example[split][i]['label'] = split_to_raw_example[split][i]['label'].replace('°', '.').split('.')[0]
    all_labels.add(split_to_raw_example[split][i]['inv_label'])

In [32]:
print(json_path)
json.dump(split_to_raw_example, open(json_path, 'w'), ensure_ascii=False)

## Intervention location 

In [33]:
#@title Intervention locations for all possible prompts

SPLIT_TO_INV_POSITION = {}

all_prompt_templates = {p for p in WIKI_PROMPT_SPLITS}
# all_prompt_templates.update({v for vs in ALL_ATTR_TO_PROMPTS.values() for v in vs})
print(len(all_prompt_templates))

for prompt_template in all_prompt_templates:
  if prompt_template.count('%s') != 1:
    continue
  print(prompt_template)
  prompt_input = prompt_template.replace('%s', '000000', 1)
  input_ids = tokenizer(prompt_input)['input_ids']
  toks = tokenizer.batch_decode(input_ids)
  for i in range(-1, -len(toks), -1):
    # This check only works for TinyLlama/Llama2 tokenizer.
    # If you use a different tokenizer, you need to update the code below
    # based on how 000000 is tokenized or use a different placeholder.
    if toks[i] == '0' and toks[i - 1] == '0' and toks[i - 2] == '0' and toks[i - 3] == '0':
      break
  SPLIT_TO_INV_POSITION[prompt_template] = i
  print(i, list(zip([(32 - len(input_ids)) + i for i in range(len(input_ids))], toks)))

print(min(SPLIT_TO_INV_POSITION.values()))

In [34]:
json.dump(SPLIT_TO_INV_POSITION,
          open(os.path.join(DATA_DIR, instance, f'{instance}_{entity_type}_prompt_to_entity_position.json'), 'w'),
          ensure_ascii=False, indent=2)

## Create instance for MIB

In [71]:
import json
import os
import random

import datasets
from datasets import Dataset


instance = 'gemma-2-2b'
entity_type = 'city'
INPUT_MAX_LEN = 48
FEATURE_TYPES = datasets.Features({"input": datasets.Value("string"), "label": datasets.Value("string"),
                              "source_input": datasets.Value("string"), "source_label": datasets.Value("string"),
                              "inv_label": datasets.Value("string"),
                              'split': datasets.Value("string"), 'source_split': datasets.Value("string"),
                              'entity': datasets.Value("string"), 'source_entity': datasets.Value("string")})


# Load training and test datasets. 
split_to_raw_example = json.load(open(os.path.join(DATA_DIR, f'{instance}/{instance}_{entity_type}_train.json'), 'r'))
split_to_raw_example.update(json.load(open(os.path.join(DATA_DIR, f'{instance}/{instance}_{entity_type}_context_test.json'), 'r')))
split_to_raw_example.update(json.load(open(os.path.join(DATA_DIR, f'{instance}/{instance}_{entity_type}_entity_test.json'), 'r')))

In [72]:
# Prepend an extra token to avoid tokenization changes for Llama tokenizer.
# Each sequence will start with <s> _ 0
SOS_PAD = '0'
NUM_SOS_TOKENS = 3
for split in split_to_raw_example:
  for i in range(len(split_to_raw_example[split])):
    split_to_raw_example[split][i]['inv_label'] = SOS_PAD + split_to_raw_example[split][i]['inv_label']
    split_to_raw_example[split][i]['label'] = SOS_PAD + split_to_raw_example[split][i]['label']


# Load attributes (tasks) to prompt mapping.
# ALL_ATTR_TO_PROMPTS = json.load(open(os.path.join(DATA_DIR, 'base', f'ravel_{entity_type}_attribute_to_prompts.json')))

# Load prompt to intervention location mapping.
split_to_entity_pos = json.load(open(os.path.join(DATA_DIR, instance, f'{instance}_{entity_type}_prompt_to_entity_position.json')))
SPLIT_TO_INV_LOCATIONS = {
    f'{task}{split}': {'max_input_length': INPUT_MAX_LEN,
                       'inv_position': [INPUT_MAX_LEN + pos]}
    for task, pos in split_to_entity_pos.items()
    for split in ('-train', '-test', '-val', '')
}
assert(min([min(v['inv_position']) for v in SPLIT_TO_INV_LOCATIONS.values()]) > 0)


# Preprocess the dataset.
def filter_inv_example(example):
  return (example['label'] != example['inv_label'] and
          example['source_split'] in SPLIT_TO_INV_LOCATIONS and
          example['split'] in SPLIT_TO_INV_LOCATIONS)

for split in split_to_raw_example:
  random.shuffle(split_to_raw_example[split])
  split_to_raw_example[split] = list(filter(filter_inv_example, split_to_raw_example[split]))
  if len(split_to_raw_example[split]) == 0:
    print('Empty split: "%s"' % split)
# Remove empty splits.
split_to_raw_example = {k: v for k, v in split_to_raw_example.items() if len(v) > 0}
print(f"#Training examples={sum(map(len, [v for k, v in split_to_raw_example.items() if k.endswith('-train')]))}, "
      f"#Validation examples={sum(map(len, [v for k, v in split_to_raw_example.items() if k.endswith('-val')]))}, "
      f"#Test examples={sum(map(len, [v for k, v in split_to_raw_example.items() if k.endswith('-test')]))}")
split_to_dataset = {split: Dataset.from_list(
    split_to_raw_example[split], features=FEATURE_TYPES)
                    for split in split_to_raw_example}

# #Training examples=116728, #Validation examples=20516, #Test examples=22497

In [ ]:
# json.dump(split_to_raw_example, open("mib_ravel.json", "w"), indent=4)
# split_to_raw_example = json.load(open("mib_ravel.json", "r"))

In [ ]:
# Check for duplicates
from collections import Counter, defaultdict
data = split_to_raw_example["Country-train"]
input_values = [entry["input"] for entry in data]
input_counts = Counter(input_values)

for text, count in input_counts.items():
    print(f'"{text}" appears {count} times.')

In [ ]:
for key in split_to_raw_example.keys():
    display(len(split_to_raw_example[key]))

In [73]:
mib_ravel_train = []
mib_ravel_test = []
mib_ravel_val = []

for key in split_to_raw_example.keys():
    if "-train" in key:
        mib_ravel_train.extend(split_to_raw_example[key])
    if "-test" in key:
        mib_ravel_test.extend(split_to_raw_example[key])
    if "-val" in key:
        mib_ravel_val.extend(split_to_raw_example[key])

In [77]:
# Add attribute and source_attribute label to the splits
# Train
template_to_attribute = {
    template: attr for template, attr in zip(ravel_city_prompts["train"]["Template"], ravel_city_prompts["train"]["Attribute"])
}

for entry in mib_ravel_train:
    template = entry.get("split")
    if template in template_to_attribute:
        entry["attribute"] = template_to_attribute[template]

for entry in mib_ravel_train:
    template = entry.get("source_split")
    if template in template_to_attribute:
        entry["source_attribute"] = template_to_attribute[template] if template_to_attribute[template] != "" else "wikipedia"


# Test
template_to_attribute = {
    template: attr for template, attr in zip(ravel_city_prompts["test"]["Template"], ravel_city_prompts["test"]["Attribute"])
}

filtered_mib_ravel_test = []

for entry in mib_ravel_test:
    template = entry.get("split")
    source_template = entry.get("source_split")

    if template in template_to_attribute and source_template in template_to_attribute:
        entry["attribute"] = template_to_attribute[template]
        entry["source_attribute"] = (
            template_to_attribute[source_template] if template_to_attribute[source_template] != "" else "wikipedia"
        )
        filtered_mib_ravel_test.append(entry) 

mib_ravel_test = filtered_mib_ravel_test

# for entry in mib_ravel_test:
#     template = entry.get("split")
#     if template in template_to_attribute:
#         entry["attribute"] = template_to_attribute[template]

# for entry in mib_ravel_test:
#     template = entry.get("source_split")
#     if template in template_to_attribute:
#         entry["source_attribute"] = template_to_attribute[template] if template_to_attribute[template] != "" else "wikipedia"


# Val
template_to_attribute = {
    template: attr for template, attr in zip(ravel_city_prompts["val"]["Template"], ravel_city_prompts["val"]["Attribute"])
}
filtered_mib_ravel_val = []

for entry in mib_ravel_val:
    template = entry.get("split")
    source_template = entry.get("source_split")

    if template in template_to_attribute and source_template in template_to_attribute:
        entry["attribute"] = template_to_attribute[template]
        entry["source_attribute"] = (
            template_to_attribute[source_template] if template_to_attribute[source_template] != "" else "wikipedia"
        )
        filtered_mib_ravel_val.append(entry) 

mib_ravel_val = filtered_mib_ravel_val

# for entry in mib_ravel_val:
#     template = entry.get("split")
#     if template in template_to_attribute:
#         entry["attribute"] = template_to_attribute[template]

# for entry in mib_ravel_val:
#     template = entry.get("source_split")
#     if template in template_to_attribute:
#         entry["source_attribute"] = template_to_attribute[template] if template_to_attribute[template] != "" else "wikipedia"

In [ ]:
# Reformat train
grouped_train = defaultdict(list)
for entry in mib_ravel_train:
    grouped_train[entry["input"]].append(entry)

clean_ravel_train = []
for input_value, entries in grouped_train.items():
    # Take the first instance in the list
    og_instance = entries[0]
    
    # Construct the OG dictionary
    clean_entry = {
        "template": og_instance["split"],
        "prompt": og_instance["input"],
        "label": og_instance["label"],
        "entity": og_instance["entity"],
        "attribute": og_instance["attribute"],
        "prompt_template_counterfactual": {},
        "attribute_counterfactual": {},
        "wikipedia_counterfactual": {}
    }
    
    for entry in entries:
        if entry["source_attribute"] == og_instance["attribute"]:
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["prompt_template_counterfactual"] = counterfactual
        
        if entry["source_attribute"] != og_instance["attribute"] and entry["source_attribute"] != "wikipedia":
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["attribute_counterfactual"] = counterfactual
            
        if entry["source_attribute"] == "wikipedia":
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["wikipedia_counterfactual"] = counterfactual

    clean_ravel_train.append(clean_entry)


In [ ]:
# Reformat test
grouped_test = defaultdict(list)
for entry in mib_ravel_test:
    grouped_test[entry["input"]].append(entry)

clean_ravel_test = []
for input_value, entries in grouped_test.items():
    og_instance = entries[0]
    
    # Construct the OG dictionary
    try: 
        clean_entry = {
            "template": og_instance["split"],
            "prompt": og_instance["input"],
            "label": og_instance["label"],
            "entity": og_instance["entity"],
            "attribute": og_instance["attribute"],
            "prompt_template_counterfactual": {},
            "attribute_counterfactual": {},
            "wikipedia_counterfactual": {}
        }
    except KeyError:
        display(og_instance)
    
    for entry in entries:
        if entry["source_attribute"] == og_instance["attribute"]:
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["prompt_template_counterfactual"] = counterfactual
        
        if entry["source_attribute"] != og_instance["attribute"] and entry["source_attribute"] != "wikipedia":
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["attribute_counterfactual"] = counterfactual
            
        if entry["source_attribute"] == "wikipedia":
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["wikipedia_counterfactual"] = counterfactual

    clean_ravel_test.append(clean_entry)


In [ ]:
# Reformat val
grouped_val = defaultdict(list)
for entry in mib_ravel_val:
    grouped_val[entry["input"]].append(entry)

clean_ravel_val = []
for input_value, entries in grouped_val.items():
    og_instance = entries[0]
    
    # Construct the OG dictionary
    try: 
        clean_entry = {
            "template": og_instance["split"],
            "prompt": og_instance["input"],
            "label": og_instance["label"],
            "entity": og_instance["entity"],
            "attribute": og_instance["attribute"],
            "prompt_template_counterfactual": {},
            "attribute_counterfactual": {},
            "wikipedia_counterfactual": {}
        }
    except KeyError:
        display(og_instance)
    
    for entry in entries:
        if entry["source_attribute"] == og_instance["attribute"]:
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["prompt_template_counterfactual"] = counterfactual
        
        if entry["source_attribute"] != og_instance["attribute"] and entry["source_attribute"] != "wikipedia":
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["attribute_counterfactual"] = counterfactual
            
        if entry["source_attribute"] == "wikipedia":
            counterfactual = {
                "label": entry["source_label"],
                "prompt": entry["source_input"]
            }
            clean_entry["wikipedia_counterfactual"] = counterfactual

    clean_ravel_val.append(clean_entry)

In [ ]:
# TODO: test and val sets are much larger than train set

random.shuffle(clean_ravel_val)
mib_ravel_val_split = clean_ravel_val[:len(clean_ravel_val)//2]
mib_ravel_public_test = clean_ravel_val[len(clean_ravel_val)//2:]

In [103]:
clean_ravel_train = Dataset.from_list(clean_ravel_train)
mib_ravel_val_split = Dataset.from_list(mib_ravel_val_split)
mib_ravel_public_test = Dataset.from_list(mib_ravel_public_test)

mib_ravel = DatasetDict({"train": clean_ravel_train,                        
                         "val": mib_ravel_val_split,
                         "test": mib_ravel_public_test})
mib_ravel.push_to_hub("yiksiu/mib_ravel")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/828 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yiksiu/mib_ravel/commit/1852e673b33845ab818874e9cea79834ff38932e', commit_message='Upload dataset', commit_description='', oid='1852e673b33845ab818874e9cea79834ff38932e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yiksiu/mib_ravel', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yiksiu/mib_ravel'), pr_revision=None, pr_num=None)